In [1]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile
import LibEE

import pandas as pd
import numpy as np

     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 1.3MB 8.4MB/s 


In [ ]:

def Execute(x,y):
    print(LibML.GetScoresCV(x,y, LibML.RandomForestDefault(len(x.columns)), 5))
    print(LibML.GetScoresCV(x,y, LibML.XGBoostDefault(), 5))    

In [ ]:

# 단계6. 임베딩2 :  원핫 -> 머니니스 -> 가입나이 -> 금리축소 -> 임베딩_상품코드 -> 임베딩_경과기간x현재나이 -> 스케일러
x, y = LibData.GetXYBase(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetAttainedAge(x)
x = LibData.GetIrReduction(x)
x = LibEE.GetXIntersected(x, ['ElapsedMth','AttainedAge'], 'Inter_EM_AA')
x, emb = LibEE.GetXEmbApplied(3, x, y, ['ProdCode', 'Inter_EM_AA'])
scaler, x = LibUtils.ScaleStandard(x)

# 단계7. SFBS
colsSelected = ['ElapsedMth', 'Sex', 'Prem', 'PremYr', 'AccumPrem', 'ContAmt', 'StartAge', 'FundAllo03', 'IR06', 'IR09', 'IR12', 'IR14', 'Moneyness', 'AttainedAge', 'ee_ProdCode_00', 'ee_ProdCode_01', 'ee_ProdCode_02']
x2 = x[colsSelected]
print('step7'); Execute(x2,y)
# [118.87, 0.1421, 0.952] [106.04, 0.1268, 0.9769]
 

# 단계7_2. 상위4개 모델에서, 4회,3회,2회 선택된 것들
colsSelected = ['ElapsedMth','AttainedAge','Sex','ContAmt','StartAge','ee_ProdCode_00','ee_ProdCode_01','ee_ProdCode_02','Prem','PremYr','AccumPrem','Moneyness','FundVal01','FundVal02','FundVal03','IR12','IR14']
x2 = x[colsSelected]
print('step7_2'); Execute(x2,y)
# [125.83, 0.1505, 0.9509] [116.47, 0.1392, 0.9713]


# 단계7_3. 변수선택 : 나의 감... 단계7_2에서 fundAllo03 투입, fundval01삭제
colsSelected = ['ElapsedMth','AttainedAge','Sex','ContAmt','StartAge','ee_ProdCode_00','ee_ProdCode_01','ee_ProdCode_02','Prem','PremYr','AccumPrem','Moneyness','FundAllo03','FundVal02','FundVal03','IR12','IR14']
x2 = x[colsSelected]
print('step7_3'); Execute(x2,y)
# [120.42, 0.144, 0.9511] [112.07, 0.134, 0.9721]


# 단계7_4. 변수선택 : 랜덤포레스트 탑17
colsSelected = ['AccumPrem','Moneyness','AttainedAge','ee_Inter_EM_AA_01','Prem','StartAge','IR14','IR12','ee_ProdCode_01','ee_ProdCode_02','ElapsedMth','ee_Inter_EM_AA_00','ee_Inter_EM_AA_02','PremYr','Sex','ee_ProdCode_00','FundVal01']
x2 = x[colsSelected]
print('step7_4'); Execute(x2,y)
# [168.35, 0.2013, 0.9357] [159.24, 0.1904, 0.962]



# 단계7_5. 변수선택 : XGBoost 탑17
colsSelected = ['IR14','AccumPrem','ContAmt','IR12','ee_Inter_EM_AA_01','Prem','Moneyness','ee_ProdCode_01','AttainedAge','Sex','ee_ProdCode_00','StartAge','ee_Inter_EM_AA_02','PremYr','ee_ProdCode_02','ee_Inter_EM_AA_00','ElapsedMth']
x2 = x[colsSelected]
print('step7_5'); Execute(x2,y)
# [167.25, 0.2, 0.9368] [157.51, 0.1883, 0.9631]


LibUtils.ELoop()